### Tutorial 04: Saving and Loading Models in PyTorch

How to save and load models in PyTorch. PyTorch provides two primary ways to save a model:

1. **Saving the entire model**
2. **Saving only the model parameters (state_dict)**

---

#### **1. Saving the Entire Model**

This approach saves the model architecture along with its parameters, allowing you to reload the model without redefining its structure.

##### **Steps**

1. Save the model:
   ```python
   import torch

   ## Assuming `model` is your PyTorch model
   torch.save(model, 'model.pth')
   ```

2. Load the model:
   ```python
   ## Load the entire model
   model = torch.load('model.pth')
   ```

##### **Pros**
- Simple to save and load.
- No need to redefine the model class when loading.

##### **Cons**
- Less flexible if you want to modify the model architecture later.

---




#### **2. Saving Only the Model Parameters (Recommended)**

This method saves only the model's parameters (state_dict), which is a dictionary containing all the learnable parameters and buffers.

##### **Steps**

1. **Save** the model's state_dict:
   ```python
   import torch

   ## Save model parameters
   torch.save(model.state_dict(), 'model_state_dict.pth')
   ```

2. **Load** the state_dict into the model:
   ```python
   ## Recreate the model architecture
   model = MyModelClass()

   ## Load state_dict
   model.load_state_dict(torch.load('model_state_dict.pth'))

   ## Set the model to evaluation mode (if using for inference)
   model.eval()
   ```

##### **Pros**
- Flexible: You can modify the model architecture and then load the saved parameters.
- Recommended for most use cases.

##### **Cons**
- Requires redefining the model class before loading.

---

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

data = {
    "x1": [2.0, 3.5, 1.8, 2.5, 3.0],
    "x2": [3000, 4000, 2800, 3500, 3700],
    "x3": [4, 6, 4, 6, 6],
    "y": [30, 20, 35, 25, 22]
}

df = pd.DataFrame(data)


model = nn.Sequential(
    nn.Linear(3, 2),  
    nn.ReLU(),        
    nn.Linear(2, 1)   
)

X = df[["x1", "x2", "x3"]].values
y_actual = df['y'].values
X_normalized = (X - X.mean(axis=0)) / X.std(axis=0)

X_tensor = torch.tensor(X_normalized, dtype=torch.float32)
y_tensor = torch.tensor(y_actual, dtype=torch.float32).view(-1, 1)


criterion = nn.MSELoss()  
optimizer = optim.SGD(model.parameters(), lr=0.01)  
epochs = 1000

for epoch in range(epochs):
    model.train()
    y_pred = model(X_tensor)
    loss = criterion(y_pred, y_tensor)

    # Zero gradients, backward pass, optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



In [2]:
checkpoint = {
    'input_size': 3,  
    'output_size': 1,  
    'hidden_layers': [2],  
    'state_dict': model.state_dict()  
}

model_path = "./models/test_model.pth"
torch.save(checkpoint, model_path)

#### **3. Saving with the Optimizer (Optional)**

When you need to resume training later, you may also want to save the optimizer's state along with the model.

##### **Steps**

1. Save the model and optimizer:
   ```python
   import torch

   ## Save model and optimizer state_dict
   torch.save({
       'model_state_dict': model.state_dict(),
       'optimizer_state_dict': optimizer.state_dict(),
       'epoch': epoch,
       'loss': loss,
   }, 'checkpoint.pth')
   ```

2. Load the checkpoint:
   ```python
   ## Load the checkpoint
   checkpoint = torch.load('checkpoint.pth')

   ## Recreate the model and optimizer
   model = MyModelClass()
   optimizer = torch.optim.Adam(model.parameters())

   ## Load state_dicts
   model.load_state_dict(checkpoint['model_state_dict'])
   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

   ## Restore other training details
   epoch = checkpoint['epoch']
   loss = checkpoint['loss']
   ```

##### **Pros**
- Saves both model and training information.
- Useful for resuming training.

---


#### **Additional Tips**

1. **Set the model to evaluation mode for inference:**
   ```python
   model.eval()
   ```
   This disables layers like dropout and batch normalization during inference.

2. **Move the model to the appropriate device:**
   ```python
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   model.to(device)
   ```

3. **Use meaningful filenames:**
   - Include details like the epoch or loss value in the filename, e.g., `model_epoch10_loss0.02.pth`.

---

#### **Summary**

- Use `torch.save()` and `torch.load()` for saving and loading models.
- Save the entire model for simplicity, but save the state_dict for flexibility.
- Include optimizer and training details in checkpoints for resuming training.